# 경로 지정

In [3]:
# 확인할 행 번호
TARGET_ROW = 1084

In [ ]:
import pandas as pd
import numpy as np
import shutil  # 파일 복사용
import json

from pathlib import Path
from tqdm import tqdm

# -------------------------------------------------------
# 경로 설정
# -------------------------------------------------------
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data")
CSV_PATH = BASE_DIR / "metadata_yolo_final.csv"

# -------------------------------------------------------
# CSV 로드 및 행 접근
# -------------------------------------------------------
meta = pd.read_csv(CSV_PATH)

# -------------------------------------------------------
# 개별 경로 생성
# -------------------------------------------------------
FRAME_DIR = BASE_DIR / str(meta.loc[TARGET_ROW]['frame_path'])
KEYPOINTS_DIR = BASE_DIR / str(meta.loc[TARGET_ROW]['keypoints_path'])
INTERP_DIR = BASE_DIR / str(meta.loc[TARGET_ROW]['interp_json_path'])

# -------------------------------------------------------
# 존재 여부 확인 및 폴더 생성
# -------------------------------------------------------
for p in [FRAME_DIR, KEYPOINTS_DIR, INTERP_DIR]:
    if not p.exists():
        if p == INTERP_DIR:
            p.mkdir(parents=True, exist_ok=True)
            print(f"📁 생성 완료: {p}")
        else:
            print(f"❌ 없음: {p}")
    else:
        print(f"✅ 존재: {p}")

print(f"\nis_train = {meta.loc[TARGET_ROW]['is_train']}")
print(f"🎯 현재 처리 대상: {Path(FRAME_DIR).name}")


✅ 존재: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/1_FRAME/sample_data/ICU_sample_video/경사진 침대에서 몸통 돌리기
✅ 존재: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/2_KEYPOINTS/sample_data/ICU_sample_video/경사진 침대에서 몸통 돌리기
📁 생성 완료: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/4_INTERP_DATA/sample_data/ICU_sample_video/경사진 침대에서 몸통 돌리기

is_train = True
🎯 현재 처리 대상: 경사진 침대에서 몸통 돌리기


# 함수

## Frame 시각화 함수

In [ ]:
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def plot_skeleton_overlay(FRAME_INDEX: int):
    """
    지정한 프레임의 skeleton overlay를 표시 (모든 keypoints 표시, 얼굴 제외)

    Args:
        FRAME_INDEX (int): 확인할 프레임 번호 (예: 248)
    """

    # -------------------------------------------------------
    # 색상 및 keypoint 그룹 정의
    # -------------------------------------------------------
    COLOR_SK = (50, 50, 50)     # Skeleton = 짙은 회색
    COLOR_L  = (0, 0, 255)      # 왼쪽 keypoints = 파랑 (Matplotlib은 RGB 순서라 반대로)
    COLOR_R  = (255, 0, 0)      # 오른쪽 keypoints = 빨강
    COLOR_NEUTRAL = (0, 255, 0) # 중앙부 keypoints = 초록

    LEFT_POINTS  = [5,7,9,11,13,15]   # 왼쪽 관절 인덱스
    RIGHT_POINTS = [6,8,10,12,14,16]  # 오른쪽 관절 인덱스
    EXCLUDE_POINTS = [0,1,2,3,4]      # 제외할 keypoints (얼굴 부위)

    # -------------------------------------------------------
    # JSON 로드
    # -------------------------------------------------------
    json_path = KEYPOINTS_DIR / f"{FRAME_INDEX:06d}.json"
    if not json_path.exists():
        raise FileNotFoundError(f"❌ JSON 파일 없음: {json_path}")
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    keypoints = np.array(data["instance_info"][0]["keypoints"])  # (17, 2)
    skeleton = np.array(data["meta_info"]["skeleton_links"])     # 연결쌍

    # -------------------------------------------------------
    # 프레임 로드
    # -------------------------------------------------------
    frame_path = FRAME_DIR / f"{FRAME_INDEX:06d}.jpg"
    if not frame_path.exists():
        raise FileNotFoundError(f"❌ 프레임 없음: {frame_path}")
    frame = cv2.imread(str(frame_path))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # -------------------------------------------------------
    # PLOT
    # -------------------------------------------------------
    plt.figure(figsize=(8, 12))
    plt.imshow(frame)

    # Skeleton 표시
    for (i1, i2) in skeleton:
        if i1 in EXCLUDE_POINTS or i2 in EXCLUDE_POINTS:
            continue
        x1, y1 = keypoints[i1]
        x2, y2 = keypoints[i2]
        plt.plot([x1, x2], [y1, y2], color=np.array(COLOR_SK)/255.0, linewidth=2, alpha=0.8)

    # Keypoints 표시
    for i, (x, y) in enumerate(keypoints):
        if i in EXCLUDE_POINTS:
            continue
        if i in LEFT_POINTS:
            color = np.array(COLOR_L)/255.0
        elif i in RIGHT_POINTS:
            color = np.array(COLOR_R)/255.0
        else:
            color = np.array(COLOR_NEUTRAL)/255.0
        plt.scatter(x, y, color=color, s=40, edgecolors='black', linewidths=0.5)
        plt.text(x + 5, y - 5, str(i), fontsize=8, color='black')

    plt.axis("off")
    plt.title(f"Frame {FRAME_INDEX:06d} Skeleton Overlay", fontsize=14)
    plt.tight_layout()
    plt.show()


## 선형 보간 함수
단일 보간 원할시  
ex. Frame 5 -> range(5,6)

In [ ]:
def interpolate_json_keypoints(target_kp: int, interp_range: range):
    global KEYPOINTS_DIR  # 상위 스코프에서 제어되는 전역 변수 사용
    print(f"[INFO] 보간 대상 keypoint: {target_kp}, frame range: {interp_range.start}-{interp_range.stop - 1}")
    print(f"[INFO] 대상 폴더: {KEYPOINTS_DIR}")

    # -------------------------------------------------------
    # 모든 JSON 파일 로드 및 keypoint 좌표 추출
    # -------------------------------------------------------
    json_files = sorted(KEYPOINTS_DIR.glob("*.json"))                     # 폴더 내 JSON 파일 리스트
    all_kps = []                                                          # 모든 keypoints 저장 리스트

    for jf in json_files:                                                 # 각 JSON 파일 순회
        with open(jf, "r") as f:                                          # JSON 열기
            data = json.load(f)                                           # JSON 파싱
        kp = data["instance_info"][0]["keypoints"]                        # 첫 번째 인스턴스의 keypoints
        all_kps.append(kp)                                                # 리스트에 추가

    all_kps = np.array(all_kps)                                           # numpy 배열로 변환 (shape: [frame, 17, 2])
    print(f"[INFO] keypoints shape: {all_kps.shape}")

    # -------------------------------------------------------
    # 보간 대상 프레임 제외한 선형 보간 수행
    # -------------------------------------------------------
    frames = np.arange(len(all_kps))                                      # 전체 프레임 인덱스
    x_vals = all_kps[:, target_kp, 0]                                     # X 좌표
    y_vals = all_kps[:, target_kp, 1]                                     # Y 좌표

    valid_mask = np.ones_like(frames, dtype=bool)                         # 전체 True 마스크 생성
    valid_mask[interp_range.start:interp_range.stop] = False              # 보간 대상 구간만 False 처리

    interp_x = np.interp(frames, frames[valid_mask], x_vals[valid_mask])  # X좌표 선형 보간
    interp_y = np.interp(frames, frames[valid_mask], y_vals[valid_mask])  # Y좌표 선형 보간

    # -------------------------------------------------------
    # 보간 결과를 기존 JSON에 덮어쓰기
    # -------------------------------------------------------
    for i, jf in enumerate(json_files):                                   # 각 프레임 순회
        if interp_range.start <= i <= interp_range.stop - 1:              # 보간 대상 프레임만 수정
            with open(jf, "r") as f:                                      # JSON 읽기
                data = json.load(f)                                       # JSON 파싱

            # keypoint 좌표를 보간값으로 교체
            data["instance_info"][0]["keypoints"][target_kp][0] = float(interp_x[i])  # X 보간값
            data["instance_info"][0]["keypoints"][target_kp][1] = float(interp_y[i])  # Y 보간값

            # 덮어쓰기 저장 (원본 JSON 업데이트)
            with open(jf, "w") as f:
                json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"\n✅ Frame {interp_range.start}~{interp_range.stop - 1} 구간 보간 완료 및 JSON 덮어쓰기 완료.")


## Keypoints 고정 함수

In [ ]:
def freeze_keypoint(KEYPOINTS_DIR: Path, OUTPUT_PATH: Path, target_kps, ref_frame: int):
    """
    여러 keypoint를 기준 프레임(ref_frame)의 좌표로 전체 프레임에 고정.
    결과는 OUTPUT_PATH에 별도로 저장 (원본 JSON은 보존).

    Args:
        KEYPOINTS_DIR (Path): 원본 keypoints JSON 폴더 경로
        OUTPUT_PATH (Path): 수정된 JSON 저장 경로 (보통 4_INTERP_DATA)
        target_kps (int | list[int]): 고정할 keypoint 인덱스(들)
        ref_frame (int): 기준 프레임 번호 (예: 266)
    """

    # -------------------------------------------------------
    # 입력 타입 보정
    # -------------------------------------------------------
    if isinstance(target_kps, int):
        target_kps = [target_kps]

    OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

    # -------------------------------------------------------
    # 기준 프레임 로드
    # -------------------------------------------------------
    ref_path = KEYPOINTS_DIR / f"{ref_frame:06d}.json"
    if not ref_path.exists():
        raise FileNotFoundError(f"❌ 기준 프레임 {ref_frame:06d}.json 없음")

    with open(ref_path, "r", encoding="utf-8") as f:
        ref_data = json.load(f)

    ref_kps = np.array(ref_data["instance_info"][0]["keypoints"])
    freeze_coords = {kp: ref_kps[kp] for kp in target_kps}

    print(f"\n[INFO] 기준 프레임 {ref_frame}의 고정 대상 keypoints 좌표:")
    for kp, coord in freeze_coords.items():
        print(f"  - KP {kp}: {coord}")

    # -------------------------------------------------------
    # 전체 프레임 복사 후 수정
    # -------------------------------------------------------
    json_files = sorted(KEYPOINTS_DIR.glob("*.json"))
    print(f"[INFO] 총 {len(json_files)}개 프레임 처리 중...")

    for json_path in tqdm(json_files, desc=f"Freezing keypoints {target_kps}"):
        # 원본 JSON 로드
        with open(json_path, "r", encoding="utf-8") as fp:
            data = json.load(fp)

        kps = np.array(data["instance_info"][0]["keypoints"])

        # 지정된 keypoints 좌표 고정
        for kp, coord in freeze_coords.items():
            kps[kp] = coord

        data["instance_info"][0]["keypoints"] = kps.tolist()

        # 출력 경로 (4_INTERP_DATA 하위 동일 구조)
        out_file = OUTPUT_PATH / json_path.name
        with open(out_file, "w", encoding="utf-8") as fp:
            json.dump(data, fp, ensure_ascii=False, indent=2)

    print(f"\n✅ keypoints {target_kps} 전체 프레임 고정 완료 (기준 frame {ref_frame})")
    print(f"   → 출력 경로: {OUTPUT_PATH}")


## IQR 시각화 함수

## Velocity 시각화 함수

## smoothing 시각화 함수

In [ ]:
import json
import numpy as np
from tqdm import tqdm
from scipy.ndimage import gaussian_filter1d
from pathlib import Path
import shutil

def smooth_keypoints(KEYPOINTS_DIR: Path, INTERP_DIR: Path, target_kps, sigma: float = 2.0):
    """
    지정한 다수의 keypoint(target_kps)의 X,Y 좌표를 Gaussian smoothing 후 
    INTERP_DIR 경로에 JSON 복사 및 수정 저장.

    Args:
        KEYPOINTS_DIR (Path): 원본 keypoints JSON 폴더 경로
        INTERP_DIR (Path): 보간/스무딩 JSON 저장 폴더 경로
        target_kps (int | list[int]): smoothing 대상 keypoint 인덱스 또는 리스트
        sigma (float): Gaussian smoothing sigma 값 (기본=2.0)
    """

    # -------------------------------------------------------
    # 입력값 정리
    # -------------------------------------------------------
    if isinstance(target_kps, int):
        target_kps = [target_kps]

    json_files = sorted(KEYPOINTS_DIR.glob("*.json"))
    if not json_files:
        raise FileNotFoundError(f"❌ JSON 파일이 존재하지 않습니다: {KEYPOINTS_DIR}")

    INTERP_DIR.mkdir(parents=True, exist_ok=True)

    print(f"[INFO] 총 {len(json_files)}개 프레임, keypoints {target_kps} smoothing 중...")
    print(f"[INFO] 원본 폴더: {KEYPOINTS_DIR}")
    print(f"[INFO] 출력 폴더: {INTERP_DIR}")

    # -------------------------------------------------------
    # keypoint별 smoothing
    # -------------------------------------------------------
    for kp in target_kps:
        print(f"\n[INFO] ▶ Keypoint {kp} smoothing 시작 (σ={sigma})")

        # 좌표 추출
        x_values, y_values = [], []
        for jfile in json_files:
            with open(jfile, "r", encoding="utf-8") as f:
                data = json.load(f)
            kps = np.array(data["instance_info"][0]["keypoints"])
            x_values.append(kps[kp, 0])
            y_values.append(kps[kp, 1])

        x_values = np.array(x_values)
        y_values = np.array(y_values)

        # Gaussian smoothing
        x_smooth = gaussian_filter1d(x_values, sigma=sigma)
        y_smooth = gaussian_filter1d(y_values, sigma=sigma)
        print(f"[INFO] Gaussian smoothing 완료 (σ={sigma}) — KP {kp}")

        # JSON 수정 및 INTERP_DIR에 저장
        for i, jfile in enumerate(tqdm(json_files, desc=f"Smoothing KP {kp}")):
            # 원본 JSON 복사 후 수정
            dest_path = INTERP_DIR / jfile.name
            shutil.copy(jfile, dest_path)

            with open(dest_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            kps = np.array(data["instance_info"][0]["keypoints"])
            kps[kp, 0] = x_smooth[i]
            kps[kp, 1] = y_smooth[i]
            data["instance_info"][0]["keypoints"] = kps.tolist()

            with open(dest_path, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"✅ Keypoint {kp} smoothing 완료 (σ={sigma})")
        

## 영상 생성 함수

In [ ]:
import cv2
import json
import numpy as np
from pathlib import Path
from tqdm import tqdm

# ===========================================================
# 단일 Overlay 생성 함수
# ===========================================================
def create_overlay(frame_dir: str, json_dir: str, out_mp4: str, fps: int = 30,
                   kp_radius: int = 4, line_thickness: int = 2):
    """
    프레임 + keypoints JSON → overlay mp4 생성 (COCO 17kp 구조)
    - 좌우 반전 없음
    - 0~4번 keypoints 제외 (얼굴 제외)
    - L/R 색상 구분
    - 하단 안내문구 표시 ("L: Blue | R: Red")
    """

    frame_files = sorted(Path(frame_dir).glob("*.jpg"))
    if not frame_files:
        print(f"[WARN] No frames found in {frame_dir}")
        return

    out_mp4 = Path(out_mp4)
    out_mp4.parent.mkdir(parents=True, exist_ok=True)

    # 해상도 확인
    sample = cv2.imread(str(frame_files[0]))
    h, w = sample.shape[:2]
    writer = cv2.VideoWriter(str(out_mp4), cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # 색상 설정
    COLOR_SK = (50, 50, 50)   # Skeleton = 짙은 회색
    COLOR_L  = (255, 0, 0)    # 왼쪽 keypoints = 파랑
    COLOR_R  = (0, 0, 255)    # 오른쪽 keypoints = 빨강
    COLOR_NEUTRAL = (0, 255, 0) # 중앙부 keypoints = 초록

    LEFT_POINTS  = [5,7,9,11,13,15]
    RIGHT_POINTS = [6,8,10,12,14,16]

    for frame_path in tqdm(frame_files, total=len(frame_files), desc=f"{Path(frame_dir).name}", unit="frame"):
        frame = cv2.imread(str(frame_path))
        json_path = Path(json_dir) / (frame_path.stem + ".json")

        if json_path.exists():
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            if "instance_info" in data and len(data["instance_info"]) > 0:
                inst = data["instance_info"][0]
                kpts = np.array(inst["keypoints"])
                skeleton = data.get("meta_info", {}).get("skeleton_links", [])

                # Skeleton
                for i, j in skeleton:
                    if i < len(kpts) and j < len(kpts):
                        if i < 5 or j < 5:
                            continue
                        pt1, pt2 = tuple(map(int, kpts[i])), tuple(map(int, kpts[j]))
                        cv2.line(frame, pt1, pt2, COLOR_SK, line_thickness)

                # Keypoints
                for idx, (x, y) in enumerate(kpts):
                    if idx < 5 or x <= 0 or y <= 0:
                        continue
                    if idx in LEFT_POINTS:
                        color = COLOR_L
                    elif idx in RIGHT_POINTS:
                        color = COLOR_R
                    else:
                        color = COLOR_NEUTRAL
                    cv2.circle(frame, (int(x), int(y)), kp_radius, color, -1)

                # 안내 문구
                legend_text = "L: Blue   |   R: Red"
                cv2.putText(frame, legend_text, (20, h - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)

        frame = cv2.flip(frame, 1)  # 좌우 반전 복원
        writer.write(frame)

    writer.release()
    print(f"✅ Overlay 완료 → {out_mp4}")
